In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [12]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
clova_ocr_key = config['NAVER']['CLOVA_OCR_KEY']

os.environ.update({'CLOVA_OCR_KEY': clova_ocr_key})

In [3]:
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
from matplotlib import pyplot as plt
 
import uuid
import json
import time
import cv2
import requests

In [4]:
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)
 
    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []
 
            for i in range(len(img)):
                titles.append(title)
 
        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
 
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
 
        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [5]:
def put_text(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_coverted)
 
    if platform.system() == 'Darwin':
        font = 'AppleGothic.ttf'
    elif platform.system() == 'Windows':
        font = 'malgun.ttf'
        
    image_font = ImageFont.truetype(font, font_size)
    font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)
 
    draw.text((x, y), text, font=image_font, fill=color)
    
    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)
 
    return opencv_image

In [13]:
clova_ocr_key = os.getenv('CLOVA_OCR_KEY')
api_url = "https://40s1tdjs0r.apigw.ntruss.com/custom/v1/23456/aa9d4447014b5f6ff61370acfd70e0f5677dc1674c95fde605a9b627e10f70f7/general"

In [18]:

image_file = './test_data/test20.png'
output_file = './test_data/output1.json'

In [19]:
request_json = {
    'images': [
        {
            'format': 'png',
            'name': 'demo'
        }
    ],
    'requestId': str(uuid.uuid4()),
    'version': 'V2',
    'timestamp': int(round(time.time() * 1000))
}

payload = {'message': json.dumps(request_json).encode('UTF-8')}
files = [
  ('file', open(image_file,'rb'))
]
headers = {
  'X-OCR-SECRET': clova_ocr_key
}

response = requests.request("POST", api_url, headers=headers, data = payload, files = files)

res = json.loads(response.text.encode('utf8'))
print(res)

with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(res, outfile, indent=4, ensure_ascii=False)

{'version': 'V2', 'requestId': '49ecebf1-4aee-4a7c-9191-215fcb59b2f7', 'timestamp': 1695288128100, 'images': [{'uid': '322d16119e964fbba99d23149d72f0bb', 'name': 'demo', 'inferResult': 'SUCCESS', 'message': 'SUCCESS', 'validationResult': {'result': 'NO_REQUESTED'}, 'convertedImageInfo': {'width': 640, 'height': 480, 'pageIndex': 0, 'longImage': False}, 'fields': [{'valueType': 'ALL', 'boundingPoly': {'vertices': [{'x': 61.0, 'y': 150.0}, {'x': 71.0, 'y': 150.0}, {'x': 71.0, 'y': 162.0}, {'x': 61.0, 'y': 162.0}]}, 'inferText': '0', 'inferConfidence': 0.9965, 'type': 'NORMAL', 'lineBreak': True}, {'valueType': 'ALL', 'boundingPoly': {'vertices': [{'x': 104.0, 'y': 158.0}, {'x': 562.0, 'y': 158.0}, {'x': 562.0, 'y': 177.0}, {'x': 104.0, 'y': 177.0}]}, 'inferText': '잔류농약 부적합과 연관된 농업경영체 변수는 이래 표의 음영색과 같다.', 'inferConfidence': 0.9948, 'type': 'NORMAL', 'lineBreak': True}, {'valueType': 'ALL', 'boundingPoly': {'vertices': [{'x': 43.0, 'y': 186.0}, {'x': 71.0, 'y': 186.0}, {'x': 71.0, 'y': 199